<a href="https://colab.research.google.com/github/kikiru328/enterprise_analysis/blob/main/New_clf_ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

Mounted at /content/drive


In [2]:
#  Directory 변경 (모듈이용)
%pwd
%cd '../content/notebooks'

# module import
## Data Load
from pickle5 import pickle 

## Basic module
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')


/content/drive/My Drive/Colab Notebooks


In [3]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/훈팡/새분류.pkl','rb') as file:
  df = pickle.load(file)


In [4]:
df

,성별,연령대,거주지역,제휴사,멤버십가입개수,영수증번호,고객번호,점포코드,구매일자,구매년도,구매월,구매시간,대분류명,중분류명,소분류명,구매금액,새분류
0,F,50대,경기 김포시,A,0,2757986,15148,10,2014-09-20,2014,9,20,가구/인테리어,일용잡화,위생세제,1000,비내구재
4,F,50대,경기 김포시,A,0,2757988,2043,10,2014-09-20,2014,9,13,가구/인테리어,일용잡화,위생세제,3950,비내구재
7,F,50대,경기 김포시,A,0,2757989,2231,10,2014-09-20,2014,9,13,가구/인테리어,일용잡화,위생세제,20780,비내구재
52,F,50대,인천 계양구,A,0,1254634,2168,30,2014-01-20,2014,1,15,가구/인테리어,침구/수예,L.B,492300,내구재
53,F,60세이상,경기 김포시,A,0,1254635,732,30,2014-01-20,2014,1,12,가구/인테리어,침구/수예,한실침구,198000,내구재
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19107665,F,50대,경기 김포시,B,0,7671023,3170,19,2015-07-25,2015,7,22,교육/문화용품,전구,형광램프,5900,비내구재
19107736,F,60세이상,경기 김포시,B,0,7651580,13544,19,2015-02-26,2015,2,12,교육/문화용품,필기도구,연필깎이,750,비내구재
19107737,F,60세이상,경기 김포시,B,0,7651580,13544,19,2015-02-26,2015,2,12,교육/문화용품,오피스용품,접착용품,900,비내구재
19107738,F,60세이상,경기 김포시,B,0,7651580,13544,19,2015-02-26,2015,2,12,교육/문화용품,미술용품,물감류,15800,비내구재


In [5]:
df_1 = df[(df['구매년도']==2014)&(df['구매월']>=1) & (df['구매월']<=6)] 
df_2 = df[(df['구매년도']==2014)&(df['구매월']>=7) & (df['구매월']<=12)]
df_3 = df[(df['구매년도']==2015)&(df['구매월']>=1) & (df['구매월']<=6)]
df_4 = df[(df['구매년도']==2015)&(df['구매월']>=7) & (df['구매월']<=12)]

In [8]:
t = pd.get_dummies(df_1['새분류'])
print(t.shape)
print(df_1.shape)
t

(6704092, 4)
(6704092, 17)


,내구재,명품,비내구재,준내구재
52,1,0,0,0
53,1,0,0,0
54,0,0,0,1
55,0,0,1,0
78,0,0,1,0
...,...,...,...,...
12129306,0,0,1,0
12129307,0,0,1,0
12129367,0,0,1,0
12129368,0,0,1,0


In [27]:
def ratio(df):
  test = df[['고객번호','새분류']]
  test = test.set_index('고객번호')
  A = pd.get_dummies(test['새분류']).reset_index()
  A = A.groupby('고객번호').sum().astype(int)
  Li = list(A.columns)
  A['total'] = A.sum(axis=1).astype(int)
  for i in Li:
    A[f"{i}_r"] = A[i] / A['total']
  B = A.drop(['내구재','명품','비내구재','준내구재','total'],axis=1)
  B['total_r'] = B.sum(axis=1)
  return B

In [28]:
H1_r = ratio(df_1)
H2_r = ratio(df_2)
H3_r = ratio(df_3)
H4_r = ratio(df_4)

In [29]:
H1_r

,내구재_r,명품_r,비내구재_r,준내구재_r,total_r
고객번호,,,,,
1,0.004202,0.000000,0.693277,0.302521,1.0
2,0.032448,0.000000,0.823009,0.144543,1.0
3,0.000000,0.000000,0.997126,0.002874,1.0
4,0.000000,0.000000,0.851852,0.148148,1.0
5,0.002653,0.002653,0.952255,0.042440,1.0
...,...,...,...,...,...
19356,0.000000,0.000000,1.000000,0.000000,1.0
19357,0.000000,0.000000,0.750000,0.250000,1.0
19359,0.000000,0.000000,1.000000,0.000000,1.0


In [30]:
def reName(a,df):
    cols = list(df.columns)
    for i in cols:
      df[f'{a}_{i}'] = df[i]
    A = df.drop(cols,axis=1)
    return A

H1 = reName('H1',H1_r)
H2 = reName('H2',H2_r)
H3 = reName('H3',H3_r)
H4 = reName('H4',H4_r)

In [31]:
H1

,H1_내구재_r,H1_명품_r,H1_비내구재_r,H1_준내구재_r,H1_total_r
고객번호,,,,,
1,0.004202,0.000000,0.693277,0.302521,1.0
2,0.032448,0.000000,0.823009,0.144543,1.0
3,0.000000,0.000000,0.997126,0.002874,1.0
4,0.000000,0.000000,0.851852,0.148148,1.0
5,0.002653,0.002653,0.952255,0.042440,1.0
...,...,...,...,...,...
19356,0.000000,0.000000,1.000000,0.000000,1.0
19357,0.000000,0.000000,0.750000,0.250000,1.0
19359,0.000000,0.000000,1.000000,0.000000,1.0


In [32]:
def train_df(a,b,c):
    A = pd.concat([a,b],axis=1)
    train = pd.concat([A,c],axis=1)
    return train

train = train_df(H1,H2,H3)

In [33]:
train

,H1_내구재_r,H1_명품_r,H1_비내구재_r,H1_준내구재_r,H1_total_r,H2_내구재_r,H2_명품_r,H2_비내구재_r,H2_준내구재_r,H2_total_r,H3_내구재_r,H3_명품_r,H3_비내구재_r,H3_준내구재_r,H3_total_r
고객번호,,,,,,,,,,,,,,,
1,0.004202,0.000000,0.693277,0.302521,1.0,0.061798,0.000000,0.716292,0.221910,1.0,0.012821,0.003205,0.750000,0.233974,1.0
2,0.032448,0.000000,0.823009,0.144543,1.0,0.011070,0.003690,0.815498,0.169742,1.0,0.019157,0.015326,0.750958,0.214559,1.0
3,0.000000,0.000000,0.997126,0.002874,1.0,0.000000,0.000000,0.997442,0.002558,1.0,0.000000,0.000000,1.000000,0.000000,1.0
4,0.000000,0.000000,0.851852,0.148148,1.0,0.023256,0.000000,0.761628,0.215116,1.0,0.000000,0.000000,0.908745,0.091255,1.0
5,0.002653,0.002653,0.952255,0.042440,1.0,0.015385,0.000000,0.946154,0.038462,1.0,0.000000,0.000000,0.991870,0.008130,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,0.000000,0.000000,1.000000,0.000000,1.0,0.002625,0.000000,0.997375,0.000000,1.0,0.000000,0.000000,1.000000,0.000000,1.0
19357,0.000000,0.000000,0.750000,0.250000,1.0,0.000000,0.002353,0.941176,0.056471,1.0,0.002208,0.002208,0.964680,0.030905,1.0
19359,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,0.000000,1.000000,0.000000,1.0


In [48]:
# trainlist = ['H1','H2','H3']
# c_li = list(H1_r.columns)


# trainList = []
# for col in c_li:
#   for h in trainlist:
#     trainList.append(f'{h}_{col}')

# train = train[trainList]

train = train[['H1_내구재_r',
 'H2_내구재_r',
 'H3_내구재_r',
 'H1_명품_r',
 'H2_명품_r',
 'H3_명품_r',
 'H1_비내구재_r',
 'H2_비내구재_r',
 'H3_비내구재_r',
 'H1_준내구재_r',
 'H2_준내구재_r',
 'H3_준내구재_r']]
# trainList

In [49]:
train

,H1_내구재_r,H2_내구재_r,H3_내구재_r,H1_명품_r,H2_명품_r,H3_명품_r,H1_비내구재_r,H2_비내구재_r,H3_비내구재_r,H1_준내구재_r,H2_준내구재_r,H3_준내구재_r
고객번호,,,,,,,,,,,,
1,0.004202,0.061798,0.012821,0.000000,0.000000,0.003205,0.693277,0.716292,0.750000,0.302521,0.221910,0.233974
2,0.032448,0.011070,0.019157,0.000000,0.003690,0.015326,0.823009,0.815498,0.750958,0.144543,0.169742,0.214559
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.997126,0.997442,1.000000,0.002874,0.002558,0.000000
4,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.851852,0.761628,0.908745,0.148148,0.215116,0.091255
5,0.002653,0.015385,0.000000,0.002653,0.000000,0.000000,0.952255,0.946154,0.991870,0.042440,0.038462,0.008130
...,...,...,...,...,...,...,...,...,...,...,...,...
19356,0.000000,0.002625,0.000000,0.000000,0.000000,0.000000,1.000000,0.997375,1.000000,0.000000,0.000000,0.000000
19357,0.000000,0.000000,0.002208,0.000000,0.002353,0.002208,0.750000,0.941176,0.964680,0.250000,0.056471,0.030905
19359,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000


In [56]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/1012발표도못하고수정/10월14일이후 새로추가/train/new_ratio.pkl','wb') as train_save:
  pickle.dump(train,train_save)

In [50]:
def test_df(a,b,c,d):
    A = pd.concat([a,b],axis=1)
    train = pd.concat([A,c],axis=1)
    test = pd.concat([train,d],axis=1)
    return test

test = test_df(H1,H2,H3,H4)

In [51]:
test

,H1_내구재_r,H1_명품_r,H1_비내구재_r,H1_준내구재_r,H1_total_r,H2_내구재_r,H2_명품_r,H2_비내구재_r,H2_준내구재_r,H2_total_r,H3_내구재_r,H3_명품_r,H3_비내구재_r,H3_준내구재_r,H3_total_r,H4_내구재_r,H4_명품_r,H4_비내구재_r,H4_준내구재_r,H4_total_r
고객번호,,,,,,,,,,,,,,,,,,,,
1,0.004202,0.000000,0.693277,0.302521,1.0,0.061798,0.000000,0.716292,0.221910,1.0,0.012821,0.003205,0.750000,0.233974,1.0,0.026490,0.0,0.589404,0.384106,1.0
2,0.032448,0.000000,0.823009,0.144543,1.0,0.011070,0.003690,0.815498,0.169742,1.0,0.019157,0.015326,0.750958,0.214559,1.0,0.024490,0.0,0.755102,0.220408,1.0
3,0.000000,0.000000,0.997126,0.002874,1.0,0.000000,0.000000,0.997442,0.002558,1.0,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,0.0,1.000000,0.000000,1.0
4,0.000000,0.000000,0.851852,0.148148,1.0,0.023256,0.000000,0.761628,0.215116,1.0,0.000000,0.000000,0.908745,0.091255,1.0,0.008889,0.0,0.866667,0.124444,1.0
5,0.002653,0.002653,0.952255,0.042440,1.0,0.015385,0.000000,0.946154,0.038462,1.0,0.000000,0.000000,0.991870,0.008130,1.0,0.000000,0.0,0.966292,0.033708,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,0.000000,0.000000,1.000000,0.000000,1.0,0.002625,0.000000,0.997375,0.000000,1.0,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,0.0,0.992788,0.007212,1.0
19357,0.000000,0.000000,0.750000,0.250000,1.0,0.000000,0.002353,0.941176,0.056471,1.0,0.002208,0.002208,0.964680,0.030905,1.0,0.002924,0.0,0.967836,0.029240,1.0
19359,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,0.0,1.000000,0.000000,1.0


In [55]:
# testlist = ['H1','H2','H3','H4']
# c_li = list(H1_r.columns)

# testList = []
# for col in c_li:
#   for h in testlist:
#     testList.append(f'{h}_{col}')
# test = test[testList]

test = test[['H1_내구재_r',
 'H2_내구재_r',
 'H3_내구재_r',
 'H4_내구재_r',
 'H1_명품_r',
 'H2_명품_r',
 'H3_명품_r',
 'H4_명품_r',
 'H1_비내구재_r',
 'H2_비내구재_r',
 'H3_비내구재_r',
 'H4_비내구재_r',
 'H1_준내구재_r',
 'H2_준내구재_r',
 'H3_준내구재_r',
 'H4_준내구재_r']]
test

,H1_내구재_r,H2_내구재_r,H3_내구재_r,H4_내구재_r,H1_명품_r,H2_명품_r,H3_명품_r,H4_명품_r,H1_비내구재_r,H2_비내구재_r,H3_비내구재_r,H4_비내구재_r,H1_준내구재_r,H2_준내구재_r,H3_준내구재_r,H4_준내구재_r
고객번호,,,,,,,,,,,,,,,,
1,0.004202,0.061798,0.012821,0.026490,0.000000,0.000000,0.003205,0.0,0.693277,0.716292,0.750000,0.589404,0.302521,0.221910,0.233974,0.384106
2,0.032448,0.011070,0.019157,0.024490,0.000000,0.003690,0.015326,0.0,0.823009,0.815498,0.750958,0.755102,0.144543,0.169742,0.214559,0.220408
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.997126,0.997442,1.000000,1.000000,0.002874,0.002558,0.000000,0.000000
4,0.000000,0.023256,0.000000,0.008889,0.000000,0.000000,0.000000,0.0,0.851852,0.761628,0.908745,0.866667,0.148148,0.215116,0.091255,0.124444
5,0.002653,0.015385,0.000000,0.000000,0.002653,0.000000,0.000000,0.0,0.952255,0.946154,0.991870,0.966292,0.042440,0.038462,0.008130,0.033708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,0.000000,0.002625,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.997375,1.000000,0.992788,0.000000,0.000000,0.000000,0.007212
19357,0.000000,0.000000,0.002208,0.002924,0.000000,0.002353,0.002208,0.0,0.750000,0.941176,0.964680,0.967836,0.250000,0.056471,0.030905,0.029240
19359,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000


In [57]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/1012발표도못하고수정/10월14일이후 새로추가/test/new_ratio.pkl','wb') as test_save:
  pickle.dump(test,test_save)